In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import re

def validate_capacitor_spec(specs):
    results = {}
    
    expected_types = ["Ceramic Capacitor"]
    expected_layers = ["Multilayer"]
    expected_materials = ["Ceramic"]
    expected_mounting = ["Surface Mount", "Through Hole"]
    expected_dielectrics = ["C0G", "X7R", "X5R", "Y5V"]
    expected_sizes = ["0402", "0603", "0805", "1206"]
    size_pattern = r"^\d{4}$"

    for key, value in specs:
        if key == " " or value == "" or value.lower() == "no":
            continue
        elif key in expected_types :
            results["Type"] = "✅"
        elif key in expected_layers :
            results["Layers"] = "✅"
        elif key in expected_materials :
            results["Material"] = "✅"
        elif re.match(r"^\d+V$", key) :
            results["Voltage"] = "✅"
        elif re.match(r"^\d+% \+Tol$", key) :
            results["Tolerance +"] = "✅"
        elif re.match(r"^\d+% -Tol$", key) :
            results["Tolerance -"] = "✅"
        elif key in expected_dielectrics :
            results["Dielectric Type"] = "✅"
        elif re.match(r"^\d+ppm/Cel TC$", key) :
            results["Temp Coefficient"] = "✅"
        elif re.match(r"^\d*\.?\d+(uF|nF|pF)$", key) :
            results["Capacitance"] = "✅"
        elif key in expected_mounting and value.lower() == "yes":
            results["Mounting"] = "✅"
        elif re.match(size_pattern, key) and key in expected_sizes:
            results["Size"] = "✅"
        elif key == "CHIP":
            results["Package Type"] = "✅"
        else:
            results[key] = "❌"

    return results


chrome_options = Options()
chrome_options.add_argument("--headless")
service = Service('D:\\Z2\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe')

driver = webdriver.Chrome(service=service, options=chrome_options)

url = "https://www.findchips.com/detail/mc0603n100j101ct/SPC-Multicomp#priceStockHeader"
driver.get(url)

time.sleep(5)

html_content = driver.page_source
driver.quit()

soup = BeautifulSoup(html_content, 'html.parser')
rows = soup.find_all('tr', class_='data-row')

data_list = []
spec_values = []

for row in rows:
    field_cell = row.find('td', class_='field-cell')
    value_cell = row.find('td', class_='main-part-cell')

    if field_cell and value_cell:
        field_name = field_cell.text.strip()
        value = value_cell.get('data-value', 'N/A').strip()

        # Only add if not 'no'
        if value.lower() != "no":
            spec_values.append((field_name, value))

        data_list.append({
            "field_name": field_name,
            "value": value
        })

# Debug: print specs string for validation
spec_string = ", ".join([f"{k}: {v}" for k, v in spec_values])

print("-" * 50)
print("Specs String:", spec_string)
print("-" * 50)

# Run validation
result = validate_capacitor_spec(spec_values)
print("-" * 50)
print("Validation Result:")
print(result)


--------------------------------------------------
Specs String: : , Rohs Code: Yes, Part Life Cycle Code: Active, Ihs Manufacturer: MULTICOMP PRO, Package Description: CHIP, Reach Compliance Code: unknown, ECCN Code: EAR99, HTS Code: 8532.24.00.20, Samacsys Manufacturer: Multicomp Pro, Capacitance: 0.00001 &#181;F, Capacitor Type: CERAMIC CAPACITOR, Dielectric Material: CERAMIC, JESD-609 Code: e3, Mounting Feature: SURFACE MOUNT, Multilayer: Yes, Negative Tolerance: 5%, Number of Terminals: 2, Operating Temperature-Max: 125 &#176;C, Operating Temperature-Min: -55 &#176;C, Package Shape: RECTANGULAR PACKAGE, Packing Method: TR, PAPER, 7 INCH, Positive Tolerance: 5%, Rated (DC) Voltage (URdc): 100 V, Size Code: 0603, Surface Mount: YES, Temperature Characteristics Code: C0G, Temperature Coefficient: 30ppm/Cel ppm/&#176;C, Terminal Finish: TIN OVER NICKEL, Terminal Shape: WRAPAROUND
--------------------------------------------------
--------------------------------------------------
Vali